# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

In [1]:
!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [2]:
!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [4]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [5]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2024-09-30 15:11:13--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0s      

2024-09-30 15:11:13 (96.5 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2024-09-30 15:11:14--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14747 (14K) [text/plain]
Sa

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [7]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2024, 9, 27, 15, 11, 15, 518380)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [8]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [9]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [10]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [11]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [12]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [13]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Yes, people generally liked John Wick based on the reviews provided. The movie received positive feedback for its action sequences, Keanu Reeves' performance, and overall entertainment value."

In [14]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'"

In [15]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. It leads to a violent and gripping story with plenty of action, shootouts, and breathtaking fights.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [16]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [17]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [18]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'It seems like opinions on the John Wick movies are divided. Some people really enjoyed them, while others did not. So, it is difficult to say whether people generally liked John Wick.'

In [19]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"I'm sorry, there are no reviews with a rating of 10 based on the provided context."

In [20]:
bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the main character, John Wick, is a retired hitman seeking vengeance for the killing of his dog, which was a final gift from his deceased wife. The movie is known for its intense action sequences and choreography.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [21]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [22]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [23]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick.'

In [24]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n- Review 1: /review/rw4854296/?ref_=tt_urv'

In [25]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, the main character, John Wick, is forced back into action after a mobster blows up his house and demands his help. Wick is asked to kill the mobster's sister in Rome to help him gain power in the criminal organizations. Wick completes the task but then has a contract put on him, leading to a series of intense action sequences as he seeks revenge."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [26]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [27]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [28]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Yes, people generally liked John Wick based on the reviews provided. The feedback highlighted the slickness of Keanu Reeves' performance, the brilliance in the action sequences, and the overall entertainment value of the film."

In [29]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n- /review/rw4854296/?ref_=tt_urv'

In [30]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, the main character, played by Keanu Reeves, is a retired assassin who comes out of retirement seeking revenge after someone kills his dog and steals his car. The story involves a lot of action and violence as John Wick goes on a mission to pay off an old debt and take over the Assassin's Guild. Throughout the movie, John travels to various locations like Italy, Canada, and Manhattan, killing numerous assassins along the way."

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [31]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [32]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/var/folders/kd/n70wp5t13t1ftd6d335l_ynm0000gn/T/ipykernel_17599/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [33]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [34]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [35]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [36]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Overall, people had different opinions about John Wick. Some viewers really enjoyed the movie and thought it was the best in the series, while others did not like it at all.'

In [37]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'There is a review with a rating of 10 for the movie "John Wick 3". The URL to that review is /review/rw4854296/?ref_=tt_urv.'

In [38]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, a retired assassin named John Wick comes out of retirement to seek revenge after his dog is killed and his car is stolen. He is forced to pay off an old debt by helping take over the Assassin's Guild, leading to a lot of carnage and action scenes in Italy, Canada, and Manhattan."

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [39]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [40]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [41]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, it seems that the majority of people generally liked John Wick. The reviews praise the action sequences, Keanu Reeves' performance, and the overall fun and excitement of the movie. Overall, John Wick received positive feedback from viewers."

In [42]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'."

In [43]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the movie "John Wick," an ex-hitman comes out of retirement to seek vengeance on the gangsters who killed his dog and took everything from him. The story follows John Wick as he unleashes a maelstrom of destruction against those who target him, leading to intense action sequences and a relentless vendetta.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

In [44]:
!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [45]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [46]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [47]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [48]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [49]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [50]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews provided.'

In [51]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3." Here is the URL to that review: [Review Link](https://www.imdb.com/review/rw4854296/?ref_=tt_urv)'

In [52]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, an ex-hitman comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. The story follows John Wick as he unleashes a maelstrom of destruction against those who try to chase him, leading to intense action sequences and confrontations with killers in the business. The plot revolves around revenge and the consequences of Wick's actions, making for a gripping and action-packed story."

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

### YOUR CODE HERE

#### Set up Langsmith

In [53]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Advanced_Retrieval_with_Langchain - {unique_id}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('Enter your LangSmith API key: ')

#### Generate the synthetic dataset using the SDG.ipynb notebook

In [54]:
import pandas as pd 

synthetic_testset_df = pd.read_csv("synthetic_testset.csv")
synthetic_testset_df

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,0,What is the reputation of the John Wick action...,"["": 21\nReview: He fights and bites, he fights...",The John Wick action movie franchise is known ...,simple,"[{'source': 'john_wick_4.csv', 'row': 21, 'Rev...",True
1,1,What role does Iosef's uncle play in the plot ...,"["": 5\nReview: Iosef's uncle still has John Wi...",Iosef's uncle plays a role in the plot of John...,simple,"[{'source': 'john_wick_2.csv', 'row': 5, 'Revi...",True
2,2,How does John Wick: Chapter 3 - Parabellum exp...,"["": 24\nReview: John Wick: Chapter 3 - Parabel...",John Wick: Chapter 3 - Parabellum explores the...,simple,"[{'source': 'john_wick_3.csv', 'row': 24, 'Rev...",True
3,3,What role does the Continental hotel play in t...,"["": 20\nReview: After resolving his issues wit...","The Continental hotel in New York City, owned ...",simple,"[{'source': 'john_wick_2.csv', 'row': 20, 'Rev...",True
4,4,What is the premise of the John Wick movie ser...,"["": 19\nReview: If you've seen the first John ...",The premise of the John Wick movie series revo...,simple,"[{'source': 'john_wick_2.csv', 'row': 19, 'Rev...",True
5,5,Who is the mobster that visits John Wick and a...,"["": 20\nReview: After resolving his issues wit...",Santino D'Antonio is the mobster who visits Jo...,simple,"[{'source': 'john_wick_2.csv', 'row': 20, 'Rev...",True
6,6,What aspect of 'John Wick' received praise for...,[': 19\nReview: I really don\'t understand the...,The cinematography and fight choreography of '...,simple,"[{'source': 'john_wick_1.csv', 'row': 19, 'Rev...",True
7,7,What role does the marker play in the plot of ...,"["": 20\nReview: After resolving his issues wit...",The marker in the plot of John Wick: Chapter 2...,simple,"[{'source': 'john_wick_2.csv', 'row': 20, 'Rev...",True
8,8,What event in the movie John Wick Chapter 2 le...,"["": 10\nReview: Wow what a great surprise this...",The answer to given question is not present in...,simple,"[{'source': 'john_wick_1.csv', 'row': 10, 'Rev...",True
9,9,What role do assassins play in the artistic sp...,"["": 12\nReview: If there's an equivalent of cl...",Assassins play a significant role in the artis...,simple,"[{'source': 'john_wick_2.csv', 'row': 12, 'Rev...",True


# Create a function to test each chaing using the synthetic testset

In [55]:
from datasets import Dataset
import pandas as pd 
from langsmith.wrappers import wrap_openai
from langsmith import traceable

def test_chain(testset_path: str, chain) -> Dataset:
    
    testset_df = pd.read_csv(testset_path)

    answers = []
    contexts = []

    questions = testset_df["question"].values.tolist()
    groundtruths = testset_df["ground_truth"].values.tolist()

    # Loop over the list of synthetic questions and call the chain 
    for question in questions:
        print(question)

        # Call the chain
        response = chain.invoke({"question" : question})
        print(response)
        
        # Capture the answer and context 
        answers.append(response["response"].content)
        contexts.append([context.page_content for context in response["context"]])

    response_ds = Dataset.from_dict({
        "question" : questions,
        "answer" : answers,
        "contexts" : contexts,
        "ground_truth" : groundtruths
        })

    return response_ds

In [56]:
naive_chain_ds = test_chain("synthetic_testset.csv", naive_retrieval_chain)
naive_chain_ds.to_csv("datasets/naive_dataset_ds.csv")

What is the reputation of the John Wick action movie franchise in terms of its overall tone and style?
{'response': AIMessage(content="The reputation of the John Wick action movie franchise is highly regarded for its overall tone and style. It is described as slick, violent fun with cool and stylish action sequences, making it one of the best action films in recent years. The franchise is known for its fast-paced, kinetic chaos, and expertly choreographed action scenes that keep viewers entertained. The supporting cast, including actors like Ian McShane and Willem Dafoe, adds to the film's overall appeal. The franchise has been praised for breaking the mold of traditional action films and delivering unique and stylish content that sets it apart from other movies in the genre.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 3549, 'total_tokens': 3675, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name':

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

177204

In [57]:
bm25_chain_ds = test_chain("synthetic_testset.csv", bm25_retrieval_chain)
bm25_chain_ds.to_csv("datasets/bm25_chain_ds.csv")

What is the reputation of the John Wick action movie franchise in terms of its overall tone and style?
{'response': AIMessage(content='The reputation of the John Wick action movie franchise is highly regarded for its intense action sequences, stylish visuals, and overall entertaining tone. The franchise is known for its fast-paced storytelling, impressive choreography, and strong performances by Keanu Reeves. Fans of action movies appreciate the unique blend of revenge themes, cool characters, and non-stop action that the John Wick franchise delivers.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 1587, 'total_tokens': 1662, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7a14fc67-1df7-4343-bc38-048121dabaf1-0', usage_metadata={'input_tokens': 1587, 'output_tokens': 75, 'total_tokens': 1662}), 'co

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

62792

In [58]:
contextual_compression_chain_ds = test_chain("synthetic_testset.csv", contextual_compression_retrieval_chain)
contextual_compression_chain_ds.to_csv("datasets/contextual_compression_chain_ds.csv")

What is the reputation of the John Wick action movie franchise in terms of its overall tone and style?
{'response': AIMessage(content='The reputation of the John Wick action movie franchise is highly positive in terms of its overall tone and style. It is known for being slick, violent fun with expertly choreographed action sequences and a cool, stylish swagger. The franchise is praised for its intense and entertaining action scenes, as well as the charismatic performance of Keanu Reeves as the titular character. Overall, the John Wick movies are considered remarkable and surprising, setting a standard for great action filmmaking.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 1609, 'total_tokens': 1702, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-777a033c-ce36-4db1-9c04-077d8ddee817-0', usage_

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

59506

In [59]:
multi_query_chain_ds = test_chain("synthetic_testset.csv", multi_query_retrieval_chain)
multi_query_chain_ds.to_csv("datasets/multi_query_chain_ds.csv")

What is the reputation of the John Wick action movie franchise in terms of its overall tone and style?
{'response': AIMessage(content='The reputation of the John Wick action movie franchise is generally positive, with reviewers praising its slickness, brilliant action sequences, and cool style. The franchise is known for its violent brutality, fast-paced action, and charismatic lead performance by Keanu Reeves. Fans of the franchise appreciate its unique tone and style, making it a standout in the action genre.\n\nIf you have any more specific questions about the franchise or its tone and style, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 4944, 'total_tokens': 5036, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-976a0081-c51e-4160-b566-2d70eb82d05c-0', usage_metadata={'input_

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

224053

In [60]:
parent_document_chain_ds = test_chain("synthetic_testset.csv", parent_document_retrieval_chain)
parent_document_chain_ds.to_csv("datasets/parent_document_chain_ds.csv")

What is the reputation of the John Wick action movie franchise in terms of its overall tone and style?
{'response': AIMessage(content='The reputation of the John Wick action movie franchise is highly regarded in terms of its overall tone and style. It is known for setting a standard of what action in Hollywood should be, with expertly choreographed, shot, and edited action sequences. The franchise is considered an example of great action filmmaking.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 356, 'total_tokens': 417, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-17c26b2d-9a59-4413-ae8b-b91b28180393-0', usage_metadata={'input_tokens': 356, 'output_tokens': 61, 'total_tokens': 417}), 'context': [Document(metadata={'source': 'john_wick_4.csv', 'row': 18, 'Review_Date': '17 March 2023', 'Review_

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

32290

In [61]:
ensemble_chain_ds = test_chain("synthetic_testset.csv", ensemble_retrieval_chain)
ensemble_chain_ds.to_csv("datasets/ensemble_chain_ds.csv")

What is the reputation of the John Wick action movie franchise in terms of its overall tone and style?
{'response': AIMessage(content='The John Wick action movie franchise is highly regarded for its overall tone and style. It is often described as slick, cool, violent, fun, and stylish. The action sequences are praised for their brilliance, choreography, and intensity. The film is known for its Tarantino-esque dialog, fast-paced nature, and the confident performance of Keanu Reeves as the lead character. Overall, the franchise is seen as a standout in the action genre for its unique approach and sheer entertainment value.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 5469, 'total_tokens': 5567, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-571a2e99-f0e2-4c8e-9fbc-cdc4ebe7f7ca-0', usage_metadata

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

251685

In [63]:
semantic_chain_ds = test_chain("synthetic_testset.csv", semantic_retrieval_chain)
semantic_chain_ds.to_csv("datasets/semantic_chain_ds.csv")

What is the reputation of the John Wick action movie franchise in terms of its overall tone and style?
{'response': AIMessage(content='The John Wick action movie franchise is highly regarded for its overall tone and style, with reviewers praising its slickness, violence, and stylish action sequences. The franchise is known for its kinetic, concise, and stylish approach to action filmmaking, setting a standard for what action in Hollywood should be. The films are described as being fun, violent, and entertaining, with expertly choreographed, shot, and edited action scenes that keep viewers engaged. Overall, the reputation of the John Wick franchise in terms of its tone and style is very positive.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 3138, 'total_tokens': 3247, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logpr

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

128814

#### Run Ragas evaluations on the retrieval chain datasets using metrics focused on retrieval

In [71]:
from ragas import evaluate
from ragas.metrics import (
    context_recall,
    context_precision,
)

# Context precision measures the signal-to-noise ratio of the retrieved context. This metric is computed using the question and the contexts.
# Context recall measures if all the relevant information required to answer the question was retrieved. This metric is computed based on the ground_truth 

metrics = [
    context_recall,
    context_precision,
]

/opt/anaconda3/envs/aie4/lib/python3.11/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/anaconda3/envs/aie4/lib/python3.11/site-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.metrics._answer_correctness import AnswerCorrectness, answer_correctness
/op

In [72]:
datasets = [naive_chain_ds,
         bm25_chain_ds,
         contextual_compression_chain_ds,
         multi_query_chain_ds,
         parent_document_chain_ds,
         ensemble_chain_ds,
         semantic_chain_ds,
         ]

In [77]:
ragas_results = []

for dataset in datasets:
    ragas_result = evaluate(dataset, metrics)
    ragas_results.append(ragas_result)

Evaluating:   0%|          | 0/38 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/38 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/38 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/38 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/38 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/38 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/38 [00:00<?, ?it/s]

In [80]:
for ragas_result in ragas_results:
    print(ragas_result)

{'context_recall': 0.8202, 'context_precision': 0.8404}
{'context_recall': 0.6886, 'context_precision': 0.6506}
{'context_recall': 0.8026, 'context_precision': 0.8772}
{'context_recall': 0.8246, 'context_precision': 0.7323}
{'context_recall': 0.7237, 'context_precision': 0.8684}
{'context_recall': 0.9342, 'context_precision': 0.7280}
{'context_recall': 0.7991, 'context_precision': 0.6886}
